In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu May 16 09:33:50 2019

@author: ruanke
"""
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#import re
import time, os
import pandas as pd


In [2]:
def run_time(func):
    print(1)
    def new_fun(*args, **kwargs):
        t0 = time.time()
        print('开始运行程序！')
        back = func(*args, **kwargs)
        run_time = time.time() - t0
        print('函数运行时间：%d分%.2f秒' % (run_time//60, run_time % 60))
        return back
    return new_fun


In [3]:
def InitDriver():
    print(2)
    #初始化游览器选项：
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument("--test-type")
    
    #打开Chrome游览器，并窗口最大化：
    driver = webdriver.Chrome(r'C:\Users\rk\Desktop\chromedriver.exe', chrome_options=options)
    driver.implicitly_wait(10)
    driver.maximize_window()
    
    #打开网址，并给其5秒加载时间：
    driver.get(url)
    time.sleep(5)
    return driver


In [4]:
def Login(driver):
    print(3)
    #关闭最下方的广告栏，防止影响后续操作
    try:
        driver.find_element(By.XPATH, '//*[@id="tyc_banner_close"]').click()
        time.sleep(1)
    except:
        pass
    #点击登录/注册按钮
    driver.find_element_by_xpath('//*[@id="web-content"]/div/div[1]/div[1]/div/div/div[2]/div/div[5]/a').click()
    time.sleep(1)
    
    #点击密码登录
    LI = driver.find_element_by_xpath('//*[@id="dialog_06742228"]/div/div[2]/div/div/div[3]/div[3]')
    LI.find_element_by_xpath('div[1]/div[2]').click()
    time.sleep(0.5)
    
    #输入手机号、密码，并点击登录
    LI.find_element_by_xpath('div[2]/div[2]/input').send_keys(mbn)
    time.sleep(0.5)
    LI.find_element_by_xpath('div[2]/div[3]/input').send_keys(psw)
    time.sleep(0.5)
    LI.find_element_by_xpath('div[2]/div[5]').click()
    #需要手动验证！！


In [5]:
def Input_name(driver, Inputing, Company_code, Company_name):
    print(4)
    Inputing.send_keys(int(Company_code))
    time.sleep(1.5)
    
    #查询该公司，并打开网址
    Search_list = driver.find_elements_by_xpath('//*[@id="web-content"]/div/div[1]/div[2]/div/div/div[2]/div[2]/div[1]/form/div/ul/li')
    for search in Search_list:
        if Company_name in search.text:
            search.click()
            time.sleep(1)
            break
    
    #如果公司代码无效，则输入公司名：
    if len(driver.window_handles) != 2:
        for j in range(30):
            Inputing.send_keys(Keys.BACKSPACE)
        
        Inputing.send_keys(Company_name)
        time.sleep(1.5)
        
        Search_list = driver.find_elements_by_xpath('//*[@id="web-content"]/div/div[1]/div[2]/div/div/div[2]/div[2]/div[1]/form/div/ul/li')
        for search in Search_list:
            if Company_name in search.text:
                search.click()
                time.sleep(1)
                break


In [6]:
def get_position(driver,chair_name,CEO_name):
    print(5)
    chair_edu, CEO_edu, CFO_name, CFO_pos, CFO_edu = '未找到','未找到','未找到','未找到','未找到'
    while True:
        #定位高管成员元素以及更改页面元素：
        Senior_people = driver.find_elements_by_xpath('//*[@id="_container_seniorPeople"]/table/tbody/tr')
        Change_page = driver.find_elements_by_xpath('//*[@id="_container_seniorPeople"]/div/ul/li')
        
        #遍历高管成员列表，寻找高管学历及财务负责人信息
        for people in Senior_people:
            if '财务' in people.find_element_by_xpath('td[3]').text:
                try:
                    CFO_name = people.find_element_by_xpath('td[2]/table/tbody/tr/td[2]/a').text
                except:
                    CFO_name = people.find_element_by_xpath('td[2]/table/tbody/tr/td[2]/span').text
                CFO_pos = people.find_element_by_xpath('td[3]').text
                CFO_edu = people.find_element_by_xpath('td[6]').text
            if chair_name == people.find_element_by_xpath('td[2]/table/tbody/tr/td[2]').text:
                chair_edu = people.find_element_by_xpath('td[6]').text
            if CEO_name == people.find_element_by_xpath('td[2]/table/tbody/tr/td[2]').text:
                CEO_edu = people.find_element_by_xpath('td[6]').text
        
        time.sleep(2)
        if Change_page[-1].text == '>':
            driver.execute_script('arguments[0].scrollIntoView(false);', Change_page[-1])
            wait = WebDriverWait(driver, 10)
            wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="_container_seniorPeople"]/div/ul/li[%s]'%len(Change_page)))).click()
            #Change_page[-1].click()
            time.sleep(2)
        else:
            return chair_edu, CEO_edu, CFO_name, CFO_pos, CFO_edu


In [7]:
@run_time
def Finding1(driver, sub_indexs):
    print(6)
    #读取数据，即需要搜索的交易所信息；创建报错指数列表
    #列名：0：公司代码、1：公司全称、2：董事长、3：最高学历1、4：CEO、
    #     5：最高学历2、6：CFO、7：财务职称、8：最高学历3、9：公司官网
    #     10：天眼查网页
    data = pd.read_excel(os.path.join(filepath, filename + '.xlsx')).fillna('未找到')
    error_indexs = []
    
    #定位搜索框
    Inputing = driver.find_element_by_xpath('//*[@id="home-main-search"]')
    time.sleep(1)
    
    #遍历整个Dataframe
    for i in sub_indexs:
        #清楚搜索框数据
        for j in range(30):
            Inputing.send_keys(Keys.BACKSPACE)
        
        #修改公司代码为字符串，例如263 -> 000263
        data.iloc[i,0] = str(data.iloc[i,0])
        for j in range(6-len(str(data.iloc[i,0]))):
            data.iloc[i,0] = '0' + str(data.iloc[i,0])
        
        #输入公司名代码
        Company_code = data.iloc[i, 0]
        Company_name = data.iloc[i, 1]
        print('正在抓取第{}个公司的数据：\n{}'.format(i, Company_name))
        
        #输入公司代码或者公司名：
        Input_name(driver, Inputing, Company_code, Company_name)
        
        #进入公司页面，若切换窗口失败，则报错并退出循环
        try:
            driver.switch_to.window(driver.window_handles[1])
        except Exception as e:
            print('搜索框发生错误，需要重新下载：\n%s\n' % e)
            error_indexs.append(i)
            continue
        time.sleep(1)
        
        #抓取数据，分别对应：天眼查网页、董事长、总经理
        data.iloc[i,10] = driver.current_url
        try:
            data.iloc[i,2] = driver.find_element_by_xpath('//*[@id="nav-main-stockNum"]/div[2]/table/tbody/tr[5]/td[2]').text
        except:
            data.iloc[i,2] = driver.find_element_by_xpath('//*[@id="nav-main-stockNum"]/div[2]/table/tbody/tr[5]/td[2]/a').text
        data.iloc[i,4] = driver.find_element_by_xpath('//*[@id="nav-main-stockNum"]/div[2]/table/tbody/tr[6]/td[4]/a').text
        
        #公司官网
        try:
            data.iloc[i,9] = driver.find_element_by_xpath('//*[@id="_container_corpContactInfo"]/table/tbody/tr[2]/td[4]').text
        except:
            data.iloc[i,9] = '-'
        
        time.sleep(2)
        
        #查找财其他数据
        try:
            data.iloc[i,3], data.iloc[i,5], data.iloc[i,6], data.iloc[i,7], data.iloc[i,8] = get_position(driver,data.iloc[i,2],data.iloc[i,4])
        except Exception as e:
            print('查询数据时发生错误，需要重新下载：\n%s\n' % e)
            error_indexs.append(i)
        else:
            print('数据抓取成功！\n')
        #关闭该标签页，并返回主页面
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(0.5)
        
        #删除输入的公司名
        for j in range(30):
            Inputing.send_keys(Keys.BACKSPACE)
        time.sleep(3)
    return data.loc[sub_indexs,:], error_indexs


1


In [8]:

def Save_to_excel(data, sub_indexs):
    print(7)
    top = max(sub_indexs)
    low = min(sub_indexs)
    data.to_excel(os.path.join(filepath, 'Result_'+filename+'_{}-{}'.format(low,top))+'.xlsx'
                , index = False)
    print('保存{}-{}成功！'.format(low,top))


In [9]:

#讲Indexs拆分，没each个为一个sub_index，返回这个装有sub_index的列表
def Split(Ind):
    print(8)
    Index = list(Ind)
    li_indexs = []
    num = len(Index)
    if num % each:
        step = num//each
    else:
        step = num//each - 1
    for j in range(step+1):
        sub_index = []
        if j != step:
            top = each+j*each
        else:
            top = num
        for i in range(0+j*each, top):
            sub_index.append(Index[i])
        li_indexs.append(sub_index)
    return li_indexs



In [10]:
def Download(driver, Indexs):
    print(9)
    li_indexs = Split(Indexs)
    Error_indexs = []
    for li in li_indexs:
        result, error_indexs = Finding1(driver, li)
        Error_indexs.extend(error_indexs)
        Save_to_excel(result,li)
        for item in error_indexs:
            with open(os.path.join(filepath,'Error_' + filename + '.txt'),'a+') as file:
                file.write('{}\n'.format(item))
    return Error_indexs


In [11]:
url = 'https://www.tianyancha.com/'
filepath = r'C:\Users\rk\Desktop\天眼查'
filename = '1'
each = 70
mbn = '18101898426'
psw = 'qwerty123'
Indexs = range(66)

In [ ]:
if __name__ == '__main__':
    driver = InitDriver()
    #Login(driver)
    Errors = Download(driver, Indexs)
    driver.quit()
    

In [12]:
driver = InitDriver()

2


In [13]:
Errors = Download(driver, Indexs)

9
8
开始运行程序！
6
正在抓取第0个公司的数据：
亚宝药业集团股份有限公司
4
5
数据抓取成功！

正在抓取第1个公司的数据：
奥瑞德光电股份有限公司
4
5
数据抓取成功！

正在抓取第2个公司的数据：
上海现代制药股份有限公司
4
5
查询数据时发生错误，需要重新下载：
Message: no such element: Unable to locate element: {"method":"xpath","selector":"td[2]/table/tbody/tr/td[2]/a"}
  (Session info: chrome=78.0.3904.108)
  (Driver info: chromedriver=72.0.3626.7 (efcef9a3ecda02b2132af215116a03852d08b9cb),platform=Windows NT 10.0.18363 x86_64)


正在抓取第3个公司的数据：
香飘飘食品股份有限公司
4
5
查询数据时发生错误，需要重新下载：
list index out of range

正在抓取第4个公司的数据：
营口港务股份有限公司
4


KeyboardInterrupt: 

In [20]:
data

NameError: name 'data' is not defined